**Imports & Setup**

In [2]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers, Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications import resnet50

2025-11-10 14:37:59.807635: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.


In [3]:
TRAIN_DIR = Path("../../data/processed_balanced/train")
IMG_DIR = Path("../../data/processed")
VAL_DIR = IMG_DIR / "validate"
TEST_DIR = IMG_DIR / "test"
EXTS = (".jpg", ".jpeg", ".png", ".bmp")
RESULTS_DIR = Path("./results")
RESULTS_PATH = RESULTS_DIR / "cnn_results.csv"
EXTS = (".jpg", ".jpeg", ".png", ".bmp")

# target styles for classification
TARGET_STYLES = [
    "Abstract_Expressionism",
    "Baroque",
    "Cubism",
    "Impressionism",
    "Pop_Art"
]

RANDOM_SEED = 635

**Load Data & Normalize**

In [4]:
# config for data loading and feature extraction
BATCH = 64
AUTO = tf.data.AUTOTUNE
EXPECT_SIZE = (256, 256)
STRICT_SIZE = True

# map from class name to label index
CLASS_TO_IDX = {c: i for i, c in enumerate(TARGET_STYLES)}
def list_paths_labels(root: Path):
    paths, labels = [], []
    for class_name in TARGET_STYLES:
        class_dir = root / class_name
        if not class_dir.exists():
            continue
        for path in sorted(class_dir.glob(f"*")):
            if path.suffix.lower() in EXTS and path.is_file():
                paths.append(str(path))
                labels.append(CLASS_TO_IDX[class_name])
    return np.array(paths, dtype=np.str_), np.array(labels, dtype=np.int32)

def decode_keep_size(path):
    img = tf.io.read_file(path)
    img = tf.image.decode_image(img, channels=3, expand_animations=False)
    img.set_shape([None, None, 3])
    if STRICT_SIZE:
        shape = tf.shape(img)
        assert_op = tf.debugging.assert_equal(
            shape[:2], EXPECT_SIZE,
            message="Non-224 image found"
        )
        with tf.control_dependencies([assert_op]):
            img = tf.identity(img)
    img = tf.cast(img, tf.float32)
    img = resnet50.preprocess_input(img)
    return img

def build_dataset(paths, labels=None, shuffle=False):
    x = tf.data.Dataset.from_tensor_slices(paths)
    x = x.map(decode_keep_size, num_parallel_calls=AUTO)
    if labels is not None:
        y = tf.data.Dataset.from_tensor_slices(labels)
        ds = tf.data.Dataset.zip((x, y))
    else:
        ds = x
    if shuffle and len(paths) > 1:
        ds = ds.shuffle(buffer_size=min(10000, len(paths)), seed=42, reshuffle_each_iteration=False)
    ds = ds.batch(BATCH).prefetch(AUTO)
    return ds

# load ResNet50 model
backbone = resnet50.ResNet50(include_top=False, weights="imagenet", pooling="avg")
train_paths, y_train = list_paths_labels(TRAIN_DIR)
val_paths, y_val = list_paths_labels(VAL_DIR)
test_paths, y_test = list_paths_labels(TEST_DIR)

# build datasets
train_ds = build_dataset(train_paths, y_train, shuffle=True)
val_ds = build_dataset(val_paths, y_val)
test_ds = build_dataset(test_paths, y_test)

# extract features
X_train = backbone.predict(train_ds)
X_val = backbone.predict(val_ds)
X_test = backbone.predict(test_ds)
print("Shapes:", X_train.shape, X_val.shape, X_test.shape)

E0000 00:00:1762803488.021838   12446 cuda_executor.cc:1309] INTERNAL: CUDA Runtime error: Failed call to cudaGetRuntimeVersion: Error loading CUDA libraries. GPU will not be used.: Error loading CUDA libraries. GPU will not be used.
W0000 00:00:1762803488.026643   12446 gpu_device.cc:2342] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step
235/235 ━━━━━━━━━━━━━━━━━━━━ 466s 2s/step
75/75 ━━━━━━━━━━━━━━━━━━━━ 147s 2s/step
75/75 ━━━━━━━━━━━━━━━━━━━━ 147s 2s/step
Shapes: (15000, 2048) (4760, 2048) (4761, 2048)


**Regularized CNN**

**Export Results & Visuals**